### Read the different datasets

In [1]:
import pandas as pd
from pathlib import Path
import yaml
import re
import numpy as np
from bokeh.io import output_notebook
import panel as pn
from bokeh.io import output_notebook
output_notebook()
pn.extension()

Loading BokehJS ...

In [2]:
def get_config():
    with open("config.yaml","r") as stream:
        config = yaml.safe_load(stream)
    return config

config = get_config()
data_dir = config["datadir"]

#The data contains some missing values, a row named Periode. This refers to the year. However,
# since the years are used as an index this rows will only contain NaN values and is thus removed.
cancer_cases = pd.read_excel(Path(data_dir + "NKR-export-data-14_01_2022_13_38_53.xlsx"), 
                             header=0, skiprows=8, index_col=0)

#The following code removes the first NaN column in windows. Somehow, drop.na() works on the linux systems, but not
#on my home computer.
cancer_cases = cancer_cases.reset_index()
cancer_cases = cancer_cases.set_index("Geografie")
cancer_cases = cancer_cases.iloc[1:,:]
cancer_cases = cancer_cases.reset_index()
cancer_cases = cancer_cases.rename(columns={"Geografie": 'Year'})
cancer_cases = cancer_cases.set_index("Year")

#Only specific columns are used. All  other columns are excluded while 
poverties = pd.read_excel(Path(data_dir + "regionaal_gemeente_pc4.ods"), 
                          engine="odf", sheet_name=1, usecols=["jaar", "gm_naam", "p_totaal", "p_uitkering"])


#Two datasets of municipalities are needed so that for all but one municipality can be searched in which province
#the municipality is located.
municipalities2010 = pd.read_excel(Path(data_dir + "2010-gemeenten-per-provincie.xls"), header=0)

municipalities2018 = pd.read_excel(Path(data_dir + "2018-gemeenten-per-provincie.xls"), header=0)

c:\users\rik\appdata\local\programs\python\python38\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [3]:
#The first thing that needs to be done is to add the province to a column in poverties. 
#The poverty data is given per province, and not per municipality.
#Therefore, instead of municipality, the poverty data needs to be viewed on a municipality level.
#This also means that the percentages of poverty per municipality have to be averaged out, to obtain the information
#per province. The average poverty percentage of each province will be calculated. 
def find_municipality(x):
    if x in municipalities2010.Gemcodel.values:
        return "".join(municipalities2010[municipalities2010.Gemcodel == x].provcodel.values)
    
    elif x in municipalities2018.Gemeentenaam.values:
        return "".join(municipalities2018[municipalities2018.Gemeentenaam == x].Provincienaam.values)
    
    #Menameradiel is the only municipality not in either of both dataframes. It is not exactly clean, but
    #returning the province in this way works perfectly.
    elif x == "Menameradiel":
        return "Friesland"

#Searches for the province the municipalties are located using the find_municipality function.
poverties["Provincie"] = list(map(find_municipality, poverties.gm_naam))

In [5]:
provinces_cancer = poverties.groupby(by=[poverties.jaar, 
                         poverties.Provincie]).agg({'p_totaal':'mean', 'p_uitkering':'mean'}).reset_index()

,jaar,Provincie,p_totaal,p_uitkering
0,2011,Drenthe,0.052750,0.205667
1,2011,Flevoland,0.065167,0.217667
2,2011,Friesland,0.056917,0.212727
3,2011,Gelderland,0.046648,0.178679
4,2011,Groningen,0.058565,0.204565
5,2011,Limburg,0.055485,0.192848
6,2011,Noord-Brabant,0.045125,0.166656
7,2011,Noord-Holland,0.049792,0.173583
8,2011,Overijssel,0.049200,0.184080
9,2011,Utrecht,0.043846,0.171769


In [6]:
def to_str(var): 
    return str(list(np.reshape(np.asarray(var), (1, np.size(var)))[0]))[1:-1]

def add_cancer_cases(x, y):
    return to_str(cancer_cases[(cancer_cases.index == str(x))][str(y)].values)


provinces_cancer["Cancer_Incidents"] = list(map(add_cancer_cases, provinces_cancer.jaar, provinces_cancer.Provincie))
provinces_cancer.Cancer_Incidents = pd.to_numeric(provinces_cancer.Cancer_Incidents).astype("int64")
#Since the Poverty and Social assitance values are given per municipality instead of Province, the mean of
#all municipalites in a province is calculated for both values.
provinces_cancer = provinces_cancer.rename(columns={"jaar":"Year", "Provincie":"Province",
                                "p_totaal":"P(Poverty)", "p_uitkering":"P(Social Assistance)"})
provinces_cancer

,Year,Province,P(Poverty),P(Social Assistance),Cancer_Incidents
0,2011,Drenthe,0.052750,0.205667,3522
1,2011,Flevoland,0.065167,0.217667,1835
2,2011,Friesland,0.056917,0.212727,4379
3,2011,Gelderland,0.046648,0.178679,12446
4,2011,Groningen,0.058565,0.204565,3701
5,2011,Limburg,0.055485,0.192848,7770
6,2011,Noord-Brabant,0.045125,0.166656,15176
7,2011,Noord-Holland,0.049792,0.173583,16170
8,2011,Overijssel,0.049200,0.184080,6799
9,2011,Utrecht,0.043846,0.171769,7362
